# Prepare files with external coordinates for MSM analysis of CDP simulations

## Import functions

In [17]:
import math
import numpy as np
import pandas as pd
import mdtraj
import itertools

## Functions

In [18]:
def angle_between(v1, v2):

    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)

    # calculate sign
    # unit vector that defines sign
    unit_vec_sign = unit_vector(np.array([1,1,1]))
    cross_prod = unit_vector(np.cross(unit_vec_sign,v1_u))
    sinus_vec = np.dot(cross_prod, v2_u)

    if sinus_vec >= 0:
        return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    else:
        return 2*np.pi - np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))


def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

## Constances

In [25]:
### Choose backbone atoms for the definition of the axes when calculating the peptide's normal plane.
    ### Depends on the specific CDP, exemplary for CDP 1
# Atoms for longitudinal axis
ellipsoid_atoms_longitudinal = np.array([28, 75])
# Atoms for transversal axis
ellipsoid_atoms_transversal = np.array([9, 58])

#Cutoff for atom-distance
cutoff_pep = .5
cutoff_aa = .5

## Load .pdb & mdtraj

In [20]:
# Directories
simulation = 'yourtrajectoryfilename.pdb'

# Name
filename = str('youroutputfilename')

In [21]:
loaded_pdb = mdtraj.load_pdb(simulation)
topology = loaded_pdb.topology

h2o_atoms = topology.select("resname SOLV")
h2o = int(len(h2o_atoms)/3)
chcl3_atoms = topology.select("resname CHCL")
chcl3 = int(len(chcl3_atoms)/5)

h2o_traj = loaded_pdb.atom_slice(h2o_atoms)
chcl3_traj = loaded_pdb.atom_slice(chcl3_atoms)

h2o_com = mdtraj.compute_center_of_mass(h2o_traj)
chcl3_com = mdtraj.compute_center_of_mass(chcl3_traj)

axis = chcl3_com-h2o_com

## CDP Position

In [22]:
### Define molecule groups   
group_pep = range(10)
group_solv = range(10,10+h2o+chcl3)
group_chcl3 = []
group_h2o = []
for number in group_solv:
    if 'CHCL' in str(topology.residue(number)):
        group_chcl3.append(number)
    elif 'SOLV' in str(topology.residue(number)):
        group_h2o.append(number)

### Analyse peptide // CHCl<sub>3</sub> contacts        
#create array that matches one peptide residue to every chloroform residue 
pairs_chcl3 = list(itertools.product(group_pep, group_chcl3))
# generate array with, its length shoud equal the number of frames 
counted_chloroform_contact = np.zeros(loaded_pdb.n_frames)
residue_contact_chcl3 = mdtraj.compute_contacts(loaded_pdb, contacts=pairs_chcl3, scheme='closest', ignore_nonprotein=False, periodic=True, soft_min=False, soft_min_beta=20)
# the contacts are stored in residue_contact[0] with shape steps x contactpairs
# we go through the contacts step by step and count how many contacts are lower than the cutoff
for timestep in range(loaded_pdb.n_frames):
    all_distances_timestep = residue_contact_chcl3[0][timestep,:]
    # here we use the fact that False counts as 0 and True as 1
    all_contacts_chcl3 = np.sum(all_distances_timestep<cutoff_pep)
    counted_chloroform_contact[timestep] = all_contacts_chcl3     

### Analyse peptide // H<sub>2</sub>O contacts
pairs_h2o = list(itertools.product(group_pep, group_h2o))
# generate array with, its length shoud equal the number of frames 
counted_water_contact = np.zeros(loaded_pdb.n_frames)
residue_contact_h2o = mdtraj.compute_contacts(loaded_pdb, contacts=pairs_h2o, scheme='closest', ignore_nonprotein=False, periodic=True, soft_min=False, soft_min_beta=20)
# the contacts are stored in residue_contact[0] with shape steps x contactpairs
# we go through the contacts step by step and count how many contacts are lower than the cutoff
for timestep in range(loaded_pdb.n_frames):
    all_distances_timestep = residue_contact_h2o[0][timestep,:]
    # here we use the fact that False counts as 0 and True as 1
    all_contacts_h2o = np.sum(all_distances_timestep<cutoff_pep)
    counted_water_contact[timestep] = all_contacts_h2o

position = counted_chloroform_contact/(counted_water_contact+counted_chloroform_contact)

### Table Position
table_position = pd.DataFrame(position,columns=['position'])
#table_position.to_csv('yourtargetdirectory'+filename+'_position.csv',index=False)

## CDP Orientation

In [23]:
all_angles_longitudinal = np.zeros(loaded_pdb.n_frames)
all_angles_transversal = np.zeros(loaded_pdb.n_frames)
all_angles_normal = np.zeros(loaded_pdb.n_frames)

# get the corresponding coordinates and put them in the right format 
# here we select the frame, if you have a trajectory you have to loop them 
for frame in range(loaded_pdb.n_frames):

    # get the coordinates of the specified frame 
    ellipsoid_coordinates_longitudinal =  loaded_pdb.xyz[frame,ellipsoid_atoms_longitudinal,:]
    ellipsoid_coordinates_transversal =  loaded_pdb.xyz[frame,ellipsoid_atoms_transversal,:]

    # get distance vector 

    distance_vector_longitudinal = ellipsoid_coordinates_longitudinal[0] - ellipsoid_coordinates_longitudinal[1]
    distance_vector_transversal = ellipsoid_coordinates_transversal[0] - ellipsoid_coordinates_transversal[1]

    # other vectors
    peptide_normal_vector = np.cross(distance_vector_longitudinal, distance_vector_transversal)

    # return angles in radians units (normal)
    angle_rad_normal = angle_between(axis[frame], peptide_normal_vector)
    all_angles_normal[frame] = angle_rad_normal

    # return angles in radians units (longitudinal)
    angle_rad_longitudinal = angle_between(axis[frame], distance_vector_longitudinal)
    all_angles_longitudinal[frame] = angle_rad_longitudinal

    # return angles in radians units (transversal)
    angle_rad_transversal = angle_between(axis[frame], distance_vector_transversal)
    all_angles_transversal[frame] = angle_rad_transversal

all_angles_normal_cos = np.cos(all_angles_normal)
all_angles_longitudinal_cos = np.cos(all_angles_longitudinal)
all_angles_transversal_cos = np.cos(all_angles_transversal)

### Table Orientation

table_all_angles_normal_cos = pd.DataFrame(all_angles_normal_cos,columns=['all_angles_normal_cos'])
#table_all_angles_normal_cos.to_csv('yourtargetdirectory'+filename+'_orientation.csv'.csv',index=False)